## **1. The Foundation: Functions are First-Class Objects**
In Python, functions are "first-class citizens." This means you can treat them like any other variable:
- You can assign a function to a variable.
- You can pass a function as an argument to another function.
- You can return a function from another function.

In [1]:
def say_hello(name):
    return f"Hello, {name}!"

# 1. Assign a function to a variable
greet = say_hello
print(greet("Alice")) # Calls the function through the 'greet' variable

# 2. Pass a function as an argument
def process_greeting(greeter_func, person_name):
    """Takes a greeting function and a name, then prints the result."""
    print(greeter_func(person_name))

process_greeting(say_hello, "Bob")
process_greeting(greet, "Charlie")

# 3. Return a function from another function
def get_greeter(greeting_word):
    """Returns a customized greeting function."""
    def greeter_function(name):
        return f"{greeting_word}, {name}!"
    return greeter_function

good_morning_greeter = get_greeter("Good morning")
print(good_morning_greeter("David"))

Hello, Alice!
Hello, Bob!
Hello, Charlie!
Good morning, David!


## **2. Building a Decorator from Scratch**
- A decorator is essentially a function that takes another function as an argument, adds some functionality (the "decoration"), and then returns the original function (or a modified version of it).
- Let's create a decorator that times how long a function takes to run.

In [2]:
import time

def timer_decorator(func):
    """A decorator that prints the execution time of the decorated function."""
    def wrapper_function(*args, **kwargs):
        # *args and **kwargs are used so the wrapper can accept any arguments
        # that the original function 'func' might take.
        
        # 1. Code to execute BEFORE the original function
        start_time = time.time()
        
        # 2. Call the original function
        result = func(*args, **kwargs)
        
        # 3. Code to execute AFTER the original function
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds.")
        
        # 4. Return the result of the original function
        return result
    
    return wrapper_function

# Now, let's decorate a function manually
def long_running_function(n):
    """A function that takes some time to run."""
    total = 0
    for i in range(n):
        total += i
    return total

# Manually applying the decorator
timed_function = timer_decorator(long_running_function)

# Calling the new, "decorated" function
result = timed_function(1000000)
print(f"Result of long_running_function: {result}")

Function 'long_running_function' executed in 0.1197 seconds.
Result of long_running_function: 499999500000


## **3. The @ Syntax (Syntactic Sugar)**
The manual process above (timed_function = timer_decorator(long_running_function)) is a bit clunky. Python provides a much cleaner way to apply a decorator using the @ symbol. This is purely "syntactic sugar" – it does the exact same thing as the manual assignment.

In [3]:
# The Pythonic way to use a decorator
@timer_decorator
def another_long_running_function(n):
    """Another function that takes time."""
    time.sleep(n) # time.sleep() pauses execution for n seconds
    print("Function finished sleeping.")

# Now when we call this function, it's automatically decorated
another_long_running_function(2)

Function finished sleeping.
Function 'another_long_running_function' executed in 2.0004 seconds.


- The line @timer_decorator is exactly equivalent to writing another_long_running_function = timer_decorator(another_long_running_function) right after the function is defined.

## **4. Real-World Use Cases**
- **Logging:** Add logging statements before and after a function runs.
- **Authentication/Authorization:** In web frameworks, a decorator can check if a user is logged in before
allowing them to access a specific page function.
- **Caching/Memoization:** Store the results of expensive function calls and return the cached result if the same inputs occur again.
- **Timing/Performance Measurement:** As we just did.
- **Data Validation:** A decorator could check if the arguments passed to a function are of the correct type or value.

In [4]:
# Example: A simple logging decorator
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function '{func.__name__}' with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"Function '{func.__name__}' returned: {result}")
        return result
    return wrapper

@log_decorator
def add(a, b):
    return a + b

add(5, 3)

# Chaining decorators
# You can apply multiple decorators. They are applied from bottom to top.
@timer_decorator
@log_decorator
def subtract(x, y):
    time.sleep(1)
    return x - y

subtract(10, 4)
# The output will show that log_decorator's prints are inside timer_decorator's timing.

Calling function 'add' with args: (5, 3), kwargs: {}
Function 'add' returned: 8
Calling function 'subtract' with args: (10, 4), kwargs: {}
Function 'subtract' returned: 6
Function 'wrapper' executed in 1.0004 seconds.


6

## **Exercises**

**1. debug_decorator:**
- Create a decorator named debug_decorator.
- This decorator should print the function's name, the arguments it was called with (`*args`, `**kwargs`), and the value it returns.
- Apply this decorator to a simple function, like one that multiplies two numbers, and call the decorated function.
- Example Output:
  - `Calling multiply(a=5, b=3)
'multiply' returned 20
The result is: 20`

In [9]:
def debug_decorator(func):
    def wrapper(*args, **kwargs):
        # Show what function is being called and with what inputs
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")

        result = func(*args, **kwargs)

        print(f"'{func.__name__}' returned: {result}")
        return result
    return wrapper

@debug_decorator
def multiply(a, b):
    return a * b
    
output = multiply(a=5, b=3)
print(f"The final result is: {output}")

Calling multiply with args: (), kwargs: {'a': 5, 'b': 3}
'multiply' returned: 15
The final result is: 15


**2. admin_only Decorator (Simulated):**
- This exercise simulates a web framework's authorization check.
-  Create a global variable user = {"username": "Riju", "role": "guest"}.
- Write a decorator named admin_only.
- Inside the decorator's wrapper function, check if the global user dictionary has a role of "admin".
- If the role is "admin", execute the original function and return its result.
- If the role is not "admin", print an "Access Denied: Admins only." message and do not run the original function (you can return None or an error message).
- Create a function show_secret_data() that prints "This is top secret data!".
- Decorate show_secret_data() with @admin_only.
- Call show_secret_data(). Then, change the global user's role to "admin" and call it again to see the difference.

In [10]:
user = {"username": "Riju", "role": "guest"}

def admin_only(func):
    """
    Decorator to restrict function access to users with 'admin' role only.
    """
    def wrapper(*args, **kwargs):
        # 2. Check if the global user's role is "admin"
        if user["role"] == "admin":
            print(f"User '{user['username']}' ({user['role']}) has access. Running '{func.__name__}'...")
            return func(*args, **kwargs)
        else:
            print(f"Access Denied: Admins only. Current user '{user['username']}' has role '{user['role']}'.")
            return None # Or you could return an error object, raise an exception, etc.
    return wrapper

@admin_only
def show_secret_data():
    """
    A function that should only be accessible by administrators.
    """
    print("This is top secret data!")


print("--- Attempt 1: User is 'guest' ---")
show_secret_data()

print("\n--- Changing user role to 'admin' ---")
user["role"] = "admin"
print("\n--- Attempt 2: User is 'admin' ---")
show_secret_data()

print("\n--- Final User Role ---")
print(user)

--- Attempt 1: User is 'guest' ---
Access Denied: Admins only. Current user 'Riju' has role 'guest'.

--- Changing user role to 'admin' ---

--- Attempt 2: User is 'admin' ---
User 'Riju' (admin) has access. Running 'show_secret_data'...
This is top secret data!

--- Final User Role ---
{'username': 'Riju', 'role': 'admin'}
